# Document Analysis in Python
In this notebook we will cover:
- Reading document data into memory
- Creating bag of words features
- Creating smoothed tf-idf features

In [1]:
import requests
import json
from contextlib import closing

# get API key saved on hardrive
with open('../NYTimesAPI.txt') as f:
    api_key = f.read() # read in my private key (sorry, not in this repo ¯\_(ツ)_/¯ )
    
# make base URL and dictionary of get request key/values
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
payload = {'api-key': api_key, 'q':'eclipse'} # key/values for get request (look up in api, there are lots)

# Perform the actual request
with closing(requests.get(url, params=payload)) as r:
    articles = r.json()
    print(articles)

{'status': 'OK', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2017/08/21/science/solar-eclipse.html', 'keywords': [{'rank': 1, 'name': 'subject', 'isMajor': 'N', 'value': 'Eclipses'}, {'rank': 2, 'name': 'subject', 'isMajor': 'N', 'value': 'Sun'}, {'rank': 3, 'name': 'subject', 'isMajor': 'N', 'value': 'Moon'}, {'rank': 4, 'name': 'subject', 'isMajor': 'N', 'value': 'Space and Astronomy'}, {'rank': 5, 'name': 'subject', 'isMajor': 'N', 'value': 'States (US)'}, {'rank': 6, 'name': 'glocations', 'isMajor': 'N', 'value': 'United States'}], 'type_of_material': 'News', 'pub_date': '2017-08-21T09:00:24+0000', 'uri': 'nyt://article/e6fd071a-5e88-57f0-9a78-5b55967f4dd0', 'multimedia': [{'width': 600, 'legacy': {'xlargewidth': 600, 'xlarge': 'images/2017/08/22/science/22ECLIPSE-live-nashville2/22ECLIPSE-live-nashville-1503345028660-articleLarge.jpg', 'xlargeheight': 400}, 'type': 'image', 'height': 400, 'url': 'images/2017/08/22/science/22ECLIPSE-live-nashville2/22ECLIPSE-live-nas

In [ ]:
# OR we can load an example query like this
# run this block of code if you can't run anything else
import json 
with open('data/nytime.json') as f:
    articles = json.loads(f.read())
    
articles

In [2]:
# get the summary text from each article
#  lead_paragraph  is no snippet
summary_text = [x['snippet'] for x in articles['response']['docs']]
summary_text

['For the first time since 1918, an eclipse traveled across the entire United States. Where the weather cooperated, those in the path took in a remarkable show.',
 'The 2017 solar eclipse seen from up above and down below.',
 'Squeeze in among the crowds and witness the first total solar eclipse to cross the entire continental United States since 1918. An exclusive video by The New York Times in partnership with NOVA/PBS.',
 'In parts of the United States, the eclipse will occur around lunchtime. Consider planning a picnic. (At least, have a crescent-shaped cookie.)',
 'Some cruise passengers will have the ultimate soundtrack for Monday\'s solar eclipse when Bonnie Tyler sings her hit "Total Eclipse of the Heart" on board.',
 'The coming solar eclipse will be a wondrous sight — if you get into viewing position in time, the clouds cooperate and you’re ready to have your mind blown.',
 'The eclipse puts us in our place.',
 'What’s happening in the heavens is a bonanza on earth.',
 'Eclip

# Converting document data into different representations
First lets go through and count the unique words in each opening sentence (that is what the NYTimes give us for free).
- http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() # an object capable of counting words in a document!

# count_vect.fit(summary_text)
# count_vect.transform(summary_text)
bag_words = count_vect.fit_transform(summary_text)

In [4]:
print(bag_words.shape) # this is a sparse matrix
print('=========')
print(bag_words[0])

(10, 123)
  (0, 83)	1
  (0, 79)	1
  (0, 100)	1
  (0, 70)	1
  (0, 49)	2
  (0, 96)	1
  (0, 24)	1
  (0, 111)	1
  (0, 114)	1
  (0, 93)	1
  (0, 105)	1
  (0, 34)	1
  (0, 4)	1
  (0, 102)	1
  (0, 33)	1
  (0, 6)	1
  (0, 0)	1
  (0, 85)	1
  (0, 97)	1
  (0, 36)	1
  (0, 95)	4
  (0, 37)	1


# Self Test: ML01b.3: 
Do you expect the vocabulary from the articles above to be:
- A. Greater than 1M words
- B. Greater than 10,000 words
- C. Fewer than 10,000 words

In [5]:
print(len(count_vect.vocabulary_))
print(count_vect.vocabulary_)

123
{'it': 52, 'united': 105, 'entire': 34, 'from': 38, 'your': 122, 'around': 8, 'heavens': 44, 'continental': 21, 'those': 96, 'ready': 78, 'that': 94, 'wondrous': 118, 'of': 64, 'york': 120, 'shaped': 82, 'cross': 26, 'tyler': 103, 'took': 100, 'earth': 32, 'parts': 68, 'is': 51, 'witness': 117, 'crescent': 25, 'safe': 80, 'glasses': 40, 'path': 70, 'have': 42, 'hit': 47, 'by': 17, 'traveled': 102, 'an': 6, 'monday': 59, 'first': 36, 'consider': 20, '1918': 0, 'soundtrack': 90, 'time': 97, 'least': 54, 'above': 3, 'new': 61, 'total': 101, 'view': 109, 'passengers': 69, 'into': 50, 'nova': 62, 'solar': 88, 'mania': 57, 'below': 11, 'coming': 19, 'remarkable': 79, 'board': 13, 'start': 92, 'partnership': 67, 'eclipse': 33, 'viewing': 110, 'make': 56, 'sight': 84, 'bonanza': 14, 'ultimate': 104, 'cruise': 28, 'at': 9, 'sings': 86, 'will': 115, 'position': 75, 'be': 10, '99': 2, 'crowds': 27, 'so': 87, 'demand': 30, 'lunchtime': 55, 'you': 121, 'pbs': 71, 'puts': 76, 'cookie': 22, 're':

In [6]:
# we can still look at the data using an inverse transform
# but we lose the ordering of the words (after all its just a bag of wrods model)
count_vect.inverse_transform(bag_words[0])

[array(['show', 'remarkable', 'took', 'path', 'in', 'those', 'cooperated',
        'weather', 'where', 'states', 'united', 'entire', 'across',
        'traveled', 'eclipse', 'an', '1918', 'since', 'time', 'first',
        'the', 'for'],
       dtype='<U11')]

In [7]:
# now let's create a pandas API out of this
import pandas as pd

pd.options.display.max_columns = 999
df = pd.DataFrame(data=bag_words.toarray(),columns=count_vect.get_feature_names())

In [8]:
df # display the full bag of words matrix  

,1918,2017,99,above,across,among,an,and,around,at,be,below,blown,board,bonanza,bonnie,building,by,clouds,coming,consider,continental,cookie,cooperate,cooperated,crescent,cross,crowds,cruise,day,demand,down,earth,eclipse,entire,exclusive,first,for,from,get,glasses,happening,have,heart,heavens,her,here,hit,if,in,into,is,it,know,least,lunchtime,make,mania,mind,monday,need,new,nova,occur,of,on,our,partnership,parts,passengers,path,pbs,picnic,place,planning,position,puts,re,ready,remarkable,safe,seen,shaped,show,sight,since,sings,so,solar,some,soundtrack,squeeze,start,states,that,the,those,time,times,to,took,total,traveled,tyler,ultimate,united,up,us,video,view,viewing,weather,what,when,where,will,with,witness,wondrous,years,york,you,your
0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,4,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,4,0,0,1,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0
3,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,2,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,2,1
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,2,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,2,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,3,0,0,0,2,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1


In [9]:
# print out 10 most common words in our data
df.sum().sort_values()[-10:]

first       3
will        3
states      3
total       3
and         4
solar       5
to          6
in          9
eclipse    10
the        20
dtype: int64

In [10]:
# print out 10 least common words in our data
df.sum().sort_values()[:10] # small sample size means most words occur one time

new            1
pbs            1
path           1
passengers     1
parts          1
partnership    1
our            1
us             1
video          1
occur          1
dtype: int64

# TF-IDF Conversion
We have a very small sample of data, but lets covert to tf-idf for the sake of programming it. Recall that Tf-idf transformation (default in `sklearn` is):

$$ \text{tf}(t,d) = f_{td}\text{, } t\in T \text{ and } d \in D $$

$$ \text{idf}(t,d) = \log{\frac{|D|}{|n_t|}}\text{, where } n_t=d\in D \text{ with } t\in d $$

$$\text{tf-idf}(t,d)=\text{tf}(t,d) \cdot (1+\text{idf}(t,d))$$

- http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer() # an object capable of counting words in a document!

tfidf_mat = tfidf_vect.fit_transform(summary_text) # that's it! its converted!!

In [12]:
# convert to pandas to get better idea about the data
df = pd.DataFrame(data=tfidf_mat.toarray(),columns=tfidf_vect.get_feature_names())
df

,1918,2017,99,above,across,among,an,and,around,at,be,below,blown,board,bonanza,bonnie,building,by,clouds,coming,consider,continental,cookie,cooperate,cooperated,crescent,cross,crowds,cruise,day,demand,down,earth,eclipse,entire,exclusive,first,for,from,get,glasses,happening,have,heart,heavens,her,here,hit,if,in,into,is,it,know,least,lunchtime,make,mania,mind,monday,need,new,nova,occur,of,on,our,partnership,parts,passengers,path,pbs,picnic,place,planning,position,puts,re,ready,remarkable,safe,seen,shaped,show,sight,since,sings,so,solar,some,soundtrack,squeeze,start,states,that,the,those,time,times,to,took,total,traveled,tyler,ultimate,united,up,us,video,view,viewing,weather,what,when,where,will,with,witness,wondrous,years,york,you,your
0,0.192030,0.000000,0.000000,0.000000,0.225893,0.000000,0.192030,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.225893,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.100277,0.192030,0.000000,0.168003,0.168003,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.220227,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.225893,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.225893,0.000000,0.000000,0.000000,0.225893,0.000000,0.192030,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.168003,0.000000,0.365909,0.225893,0.192030,0.000000,0.000000,0.225893,0.000000,0.225893,0.000000,0.000000,0.168003,0.000000,0.00000,0.000000,0.000000,0.000000,0.225893,0.000000,0.000000,0.225893,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.350265,0.000000,0.350265,0.000000,0.000000,0.000000,0.231606,0.000000,0.000000,0.000000,0.350265,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.350265,0.00000,0.155487,0.000000,0.000000,0.000000,0.000000,0.350265,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.350265,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.207995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.141843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.350265,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.167283,0.000000,0.000000,0.000000,0.000000,0.196782,0.167283,0.130118,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.196782,0.000000,0.000000,0.000000,0.196782,0.000000,0.000000,0.000000,0.000000,0.167283,0.196782,0.000000,0.000000,0.000000,0.000000,0.00000,0.087354,0.167283,0.196782,0.146353,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.191847,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.196782,0.196782,0.000000,0.000000,0.000000,0.00000,0.196782,0.000000,0.000000,0.000000,0.196782,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.167283,0.000000,0.000000,0.116853,0.000000,0.000000,0.196782,0.000000,0.146353,0.000000,0.318754,0.000000,0.000000,0.196782,0.130118,0.000000,0.146353,0.000000,0.000000,0.000000,0.146353,0.000000,0.00000,0.196782,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.196782,0.19678

In [13]:
# print out 10 words with max tfidf, normalized by document occurrence
df.max().sort_values()[-10:]

the          0.365909
heavens      0.390150
happening    0.390150
bonanza      0.390150
earth        0.390150
to           0.442870
our          0.466320
puts         0.466320
us           0.466320
place        0.466320
dtype: float64

# Working with (a bit) more data
What if we do not have the memory to deal with dense matrix representatioan and we need to keep it sparse?


In [14]:
from sklearn.datasets import fetch_20newsgroups
bunch = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)

the `bunch` object returned from sklearn is similar to a python dictionary. We can access different fields of the object with keys.

In [18]:
len(bunch.data)

11314

In [15]:
print(bunch.data[0]) # we should split this up by newlines

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [17]:
import numpy as np
#randomly grad an index
idx = round(np.random.rand()*len(bunch.data))
# and print it pretty
print("\n".join(bunch.data[idx].split("\n")))

From: himb@iniki.soest.hawaii.edu (Liz Camarra)
Subject: Some more info. about P9000 board
Originator: himb@iniki
Organization: School of Ocean and Earth Science and Technology
Lines: 10

  One more thing to add, the Orchid board vesa bios is only
able to handle the 1 meg dram on board, the Viper however can
utilize the 2 meg vram on board to support vesa modes such as
1280x1024x256, 800x600x16 mil. and 1024x768x65536 under Dos.

+----------------------------------------------------------------+
Stephen Lau, Elec. Engineering, Univ. of Hawaii
 don't have my own account until grad. school starts (autumn 93)
+              Death to FM synthesis! Go Gus!                    +




In [19]:
%%time
news_tfidf = tfidf_vect.fit_transform(bunch.data) 

CPU times: user 3.77 s, sys: 123 ms, total: 3.89 s
Wall time: 3.94 s


In [20]:
news_tfidf.shape

(11314, 130107)

In [21]:
tfidf_vect.vocabulary_

{'_1de076v5': 23140,
 'lki': 76357,
 'bockamp': 34164,
 'jzpb9t': 71309,
 '0ivbud9': 1836,
 'created': 43315,
 'c4z': 36332,
 'stasny': 110350,
 'soundly': 108969,
 'n68z': 85755,
 'thrusday': 114903,
 '1xl2': 8180,
 'zi5': 129374,
 'xtnusestringinplace': 127235,
 'merrily': 81229,
 '1tg7': 8075,
 'esprit': 52542,
 'g9261': 57828,
 '6zms': 18194,
 'tikka': 115100,
 'srmam': 109906,
 'bv90': 35884,
 'mi0mf9l3': 81755,
 'intemperate': 67718,
 '1ecr6t': 6799,
 'trencin': 116414,
 '11800': 3341,
 'fjrslwpz03hzrlj1': 55656,
 'wilt': 124384,
 'prahren': 95412,
 'c5r5vt': 36866,
 'altina': 27669,
 'kentiler': 72236,
 'j1v': 69106,
 '160493205451': 5049,
 'torum': 115794,
 '_exhausted': 23769,
 'shew': 106798,
 'snycanva': 108553,
 'sprocket': 109677,
 'shzri': 107200,
 'dg57': 46942,
 'cx_gc': 44300,
 '27u': 10161,
 '24e1d': 9616,
 'rathje': 99369,
 'zlkjrnahf': 129528,
 'nguyen': 87089,
 '7iiw': 19635,
 'crinial': 43425,
 'x5w24e': 125849,
 'forgetting': 56383,
 'fiok': 55475,
 'tanith': 113

In [22]:
# create pandas dataframe
vec = news_tfidf.max(axis=0)
df  = pd.DataFrame(data=vec.toarray(),columns=tfidf_vect.get_feature_names())

In [23]:
# largest tfidf 
df.max().sort_values()[-10:]

kk          0.870294
db          0.871473
scsi        0.875086
blah        0.879426
donoghue    0.891653
00          0.907726
___         0.908826
25          0.913127
forged      0.940511
ax          0.998314
dtype: float64

In [24]:
# now lets do the transformation with a smaller vocabulary
tfidf_vect = TfidfVectorizer(stop_words='english',
                             max_df=0.01,
                             min_df=4)
news_tfidf = tfidf_vect.fit_transform(bunch.data) 
print(news_tfidf.shape)
vec=news_tfidf.max(axis=0)
df = pd.DataFrame(data=vec.toarray(),columns=tfidf_vect.get_feature_names())
df.max().sort_values()[-10:]

(11314, 28592)


dialix            0.947270
blah              0.952881
ualberta          0.956563
stephanopoulos    0.959855
forged            0.971300
mufti             0.976947
ax                0.999881
meyers            1.000000
slower            1.000000
ucsd              1.000000
dtype: float64

# Using your own vocabulary

In [25]:
# read in scrabble dictionary from file
with open('data/ospd.txt') as f:
    vocab = f.read().split('\n')
    
# now lets do the transformation with a custom vocabulary
tfidf_vect = TfidfVectorizer(vocabulary=vocab)
news_tfidf = tfidf_vect.fit_transform(bunch.data) 
print(news_tfidf.shape)
vec=news_tfidf.max(axis=0)
df = pd.DataFrame(data=vec.toarray(),columns=tfidf_vect.get_feature_names())
df.max().sort_values()[-10:]

(11314, 79340)


incoming    0.925305
siemens     0.927114
water       0.928029
echo        0.947289
blah        0.951962
dos         0.953675
lib         0.954914
forged      0.978762
la          0.982737
ax          0.999999
dtype: float64

Looking for how to do a word cloud? Check this out:
- https://github.com/amueller/word_cloud